# Transformer all in one

In [57]:
%%capture
%load_ext autoreload
%autoreload 2

## Imports

In [58]:
import transformer_all
from transformer_all import Config, TrainHandler, DataHandler, Modules, Format

In [59]:
format = Format(style="solarized-light")
print_py = format.print_py
print_obj = format.print_obj
print_src = format.print_src

In [60]:
config=Config()
print_obj(config)

Config(seed=0,
       preview=slice(None, 10, None),
       example_item=1337,
       max_length=512,
       min_freq=5,
       default_token='<unk>',
       special_tokens=['<unk>', '<pad>'],
       batch_size=512,
       lr=0.001,
       num_epochs=20,
       heads=4,
       emb=32,
       device='cuda',
       start_time=1710128352.513732,
       last_time=0.0)



In [61]:
train_handler = TrainHandler(config=config)
print_src(train_handler.__init__)

    def __init__(self, config: Config):
        self.config = config
        np.random.seed(self.config.seed)
        torch.manual_seed(self.config.seed)
        torch.cuda.manual_seed(self.config.seed)
        torch.backends.cudnn.deterministic = True



In [62]:
data_handler = DataHandler(config=config)
print_src(data_handler.__init__)

  0.131062s   0.000000s |	DataHandler Start
  1.921671s   1.790609s |	DataHandler load_dataset
  1.921671s   0.000000s |	DataHandler get_tokenizer


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

  7.460946s   5.539275s |	DataHandler tokenize_all
 10.406100s   2.945154s |	DataHandler set_vocab


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

 21.506921s  11.100821s |	DataHandler numericalize_all
 21.507922s   0.001001s |	DataHandler format_all
 21.507922s   0.000000s |	DataHandler set_data_loaders
    def __init__(self, config: Config):
        self.config = config
        self.config.elapsed("DataHandler Start")
        self.dataset = datasets.load_dataset("zapsdcn/imdb", cache_dir="../data/classification/")
        self.config.elapsed("DataHandler load_dataset")
        self.train_dataset = self.dataset['train'] 
        self.validation_dataset = self.dataset['validation']
        self.test_dataset = self.dataset['test']
        self.tokenizer = torchtext.data.utils.get_tokenizer("basic_english")
        self.config.elapsed("DataHandler get_tokenizer")
        self.trim = slice(None,config.max_length,None)
        self.tokenize_all()
        self.config.elapsed("DataHandler tokenize_all")
        self.set_vocab()
        self.config.elapsed("DataHandler set_vocab")
        self.numericalize_all()
        self.config.elap

In [63]:
train_handler.load(data_handler=data_handler)
print_src(train_handler.load)

 21.615783s   0.107861s |	TrainHandler start
 21.621784s   0.006001s |	TrainHandler load
    def load(self, data_handler: DataHandler):
        self.config.elapsed("TrainHandler start")
        self.data_handler = data_handler
        self.model = Modules.Transformer(config=self.config, vocab_size=len(self.data_handler.vocab)).to("cuda")
        self.opt = torch.optim.Adam(lr=self.config.lr, params=self.model.parameters())
        self.config.elapsed("TrainHandler load")



In [64]:
print_src(Modules.Transformer)

    class Transformer(torch.nn.Module):
        def __init__(self, config: Config, vocab_size: int):
            super().__init__()
            self.config = config
            self.token_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=vocab_size)
            self.pos_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=self.config.max_length)
            self.tblock = Modules.TransformerBlock(config=self.config)
            self.toprobs = torch.nn.Linear(self.config.emb, 2)

        def forward(self, x):
            tokens = self.token_embedding(x)
            b, t, e = tokens.shape
            positions = self.pos_embedding(torch.arange(t, device="cuda"))[None, :, :].expand(b, t, e)
            x = tokens + positions
            x, W = self.tblock(x)
            x = torch.mean(x, dim=1)
            x = self.toprobs(x)
            return F.log_softmax(x, dim=1), W



In [65]:
train_handler.train()

 21.744336s   0.122551s |	train start
 21.757335s   0.012999s |	batch 0
 22.235862s   0.478528s |	batch 1
 22.778229s   0.542367s |	batch 2
 23.303286s   0.525056s |	batch 3
 23.841075s   0.537789s |	batch 4
 24.378634s   0.537559s |	batch 5
 24.914684s   0.536050s |	batch 6
 25.449224s   0.534540s |	batch 7
 25.988790s   0.539565s |	batch 8
 26.524840s   0.536051s |	batch 9
 27.065391s   0.540550s |	batch 10
 27.600450s   0.535060s |	batch 11
 28.135989s   0.535538s |	batch 12
 28.672540s   0.536551s |	batch 13
 29.213121s   0.540582s |	batch 14
 29.747667s   0.534546s |	batch 15
 30.284221s   0.536553s |	batch 16
 30.819185s   0.534965s |	batch 17
 31.355730s   0.536544s |	batch 18
 31.889281s   0.533551s |	batch 19
 32.426335s   0.537055s |	batch 20
 32.955868s   0.529532s |	batch 21
 33.486422s   0.530554s |	batch 22
 34.013474s   0.527053s |	batch 23
 34.542020s   0.528545s |	batch 24
 35.070562s   0.528543s |	batch 25
 35.597607s   0.527045s |	batch 26
 36.127147s   0.529541s |	b

KeyboardInterrupt: 

In [ ]:
train_handler.save()